In [2]:
from lightbm import LGBMClassifier

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classifification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split

ModuleNotFoundError: No module named 'lightbm'